In [1]:
import torch

%reload_kedro
# trained_model = catalog.load("trained_model", version="2023-03-29T11.37.00.000Z")
model_config = catalog.load("yolov8_l_description")
trained_model = torch.load("../checkpoints/checkpoint_30000.pt", map_location="cpu")

2024-05-14 09:37:23,388 - kedro.ipython - INFO - Resolved project path as: /home/daniel/git/self-supervised-learning.
To set a different path, run '%reload_kedro <project_root>'
2024-05-14 09:37:23,406 - py.warnings - WARNING - /home/daniel/git/self-supervised-learning/.venv/lib/python3.10/site-packages/kedro/framework/startup.py:99: KedroDeprecationWarning: project_version in pyproject.toml is deprecated, use kedro_init_version instead
  warnings.warn(

2024-05-14 09:37:23,543 - py.warnings - WARNING - /home/daniel/git/self-supervised-learning/.venv/lib/python3.10/site-packages/kedro/framework/session/session.py:267: KedroDeprecationWarning: TemplatedConfigLoader will be deprecated in Kedro 0.19. Please use the OmegaConfigLoader instead. To consult the documentation for OmegaConfigLoader, see here: https://docs.kedro.org/en/stable/configuration/advanced_configuration.html#omegaconfigloader
  warnings.warn(

2024-05-14 09:37:23,567 - py.warnings - WARNING - /home/daniel/git/self-superv

2024-05-14 09:37:23.835 | INFO     | self_supervised.pipelines.train_simclr.nodes:<module>:44 - Using cpu device.


In [2]:
import re
from ultralytics.models.yolo.detect.train import DetectionModel
from self_supervised.pipelines.train_simclr.moco import MoCo

# Reset lower layers in the model.
if isinstance(trained_model, MoCo):
    yolo = trained_model.encoder_q.encoder.yolo
    # yolo.model.wrapped = yolo.model._module
    # del yolo.model._module
else:
    yolo = trained_model.encoder.yolo
model_config["scale"] = "l"
new_yolo = DetectionModel(model_config)

state_dict = yolo.state_dict()
layer_num_re = re.compile(r"model\.(wrapped\.)?(\d*)\..*")
pruned_state_dict = {}
for key, params in state_dict.items():
    layer_num = int(layer_num_re.match(key).groups()[1])
    if layer_num < 10:
        pruned_state_dict[key] = params
        
incompatible_keys = new_yolo.load_state_dict(pruned_state_dict, strict=False)
assert len(incompatible_keys.unexpected_keys) == 0


                   from  n    params  module                                       arguments                     
  0                  -1  1      2240  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     74752  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    283648  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    296960  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2113536  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1183744  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8421376  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2363392  ultralytics.nn.modules.conv.Conv             [512

In [3]:
import numpy as np

assert not np.all(new_yolo.state_dict()["model.wrapped.12.m.0.cv2.conv.weight"].numpy() == yolo.state_dict()["model.wrapped.12.m.0.cv2.conv.weight"].numpy())
assert np.all(new_yolo.state_dict()["model.wrapped.9.cv2.conv.weight"].numpy() == yolo.state_dict()["model.wrapped.9.cv2.conv.weight"].numpy())

In [4]:
import torch

torch.save({"model": new_yolo}, "../yolov8l_ssl_moco.pt")